In [ ]:
import pandas as pd
import plotly.graph_objects as go
from IPython.display import display, Markdown, Image, SVG
import bmondata
import scrapbook as sb

In [ ]:
# Parameters to be changed/exported using Papermill or Scrapbook
building_id = 3
server_web_address = 'https://bmon.analysisnorth.com'

In [ ]:
if building_id != 3 or ('analysisnorth' not in server_web_address):
    raise RuntimeError('Not Applicable to this Building')

In [ ]:
sb.glue('title', 'Nighttime Nanny')
sb.glue('sort_order', 10)

# Analysis of Nighttime Power Use

The nighttime period included in this analysis is Midnight - 6 am.

In [ ]:
server = bmondata.Server(server_web_address)

In [ ]:
df = server.sensor_readings('cea_L131776533', averaging='1H')
df.columns = ['power']
df['hour'] = df.index.hour

df_night = df.query('hour >=0 and hour <= 5')
df_night_dayavg = df_night.resample('1D').mean()
df_night_dayfinal = df_night_dayavg.drop(columns=['hour']).query('power < 0.75')
df_night_dayfinal.tail()

daily_plot = go.Scatter(x=df_night_dayfinal.index, y=df_night_dayfinal.power, name='Nighttime Power')
avg_power = df_night_dayfinal.power.mean()
avg_plot = go.Scatter(
    x=[df_night_dayfinal.index[0], df_night_dayfinal.index[-1]], 
    y=[avg_power, avg_power],
    name='Nighttime Average',
    mode='lines'
)
fig = go.Figure(
    data=[daily_plot, avg_plot],
    layout_title_text = 'Daily Average Nighttime Power',
    layout_yaxis_title_text = 'Nighttime Power, kW',
)
fig.show()

In [ ]:
df_night_dayfinal['month'] = df_night_dayfinal.index.month
df_mo = df_night_dayfinal.groupby('month').mean()
plot_mo = go.Scatter(
    x=df_mo.index, 
    y=df_mo.power,
    name='Monthly Profile',
    mode='lines'
)
ser_avg_last_week = df_night_dayfinal[-8:-1].mean()
plot_last_week = go.Scatter(
    x=[ser_avg_last_week.month], 
    y=[ser_avg_last_week.power],
    name='Last Week',
    marker_size = 10
)
fig = go.Figure(
    data=[plot_mo, plot_last_week],
    layout_title_text = 'Current Week vs. Typical Monthly Nighttime Power',
    layout_xaxis_title_text = 'Month',
    layout_yaxis_title_text = 'Nighttime Power, kW'
)
fig.show()